In [1]:
# CMPT459 2020 spring
# group name: Salt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from sklearn import model_selection
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RepeatedKFold 
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

def outlier_fd(data, threshold=0.5):
    low, high = np.percentile(data,1), np.percentile(data, 99)
    dist = high - low
    cut_off = dist * threshold
    lower, upper = low-cut_off, high+cut_off
    return [lower, upper]

def photos_number_counter(x):
    return len(x)

def valid_des(s):
    valid_s = " "
    for i in s.split():
        if i.isalnum():
            valid_s = " ".join([valid_s, i])
            
    return valid_s.strip()

In [2]:
def dataforclf(train):
    #Find the outlier of price, latitude and longitude using bound given by outlier_fd
    [price_lower, price_higher] = outlier_fd(train['price'])
    outlier_price = train[(train['price']<price_lower)|(train['price']>price_higher)]
    [latitude_lower, latitude_higher] = outlier_fd(train['latitude'],1)
    outlier_latitude = train[(train['latitude']<latitude_lower)|(train['latitude']>latitude_higher)]
    [longitude_lower, longitude_higher] = outlier_fd(train['longitude'],1)
    outlier_longitude = train[(train['longitude']<longitude_lower)|(train['longitude']>longitude_higher)]
    #deal with outliers, set the column for modified data and plot again
    train['price_modify'] = train['price']
    train['price_modify'].loc[train['price_modify']<price_lower] = price_lower
    train['price_modify'].loc[train['price_modify']>price_higher] = price_higher
    #deal with outliers, set the column for modified data and plot again
    train['latitude_modify'] = train['latitude']
    train['latitude_modify'].loc[train['latitude_modify']<latitude_lower] = latitude_lower
    train['latitude_modify'].loc[train['latitude_modify']>latitude_higher] = latitude_higher
    #deal with outliers, set the column for modified data and plot again
    train['longitude_modify'] = train['longitude']
    train['longitude_modify'].loc[train['longitude']<longitude_lower] = longitude_lower
    train['longitude_modify'].loc[train['longitude']>longitude_higher] = longitude_higher
    #get the number of photos
    train['photos_number'] = train['photos'].apply(photos_number_counter)
    #set all feature words in tf_idf as features for train for feature_modify
    train['features_modify'] = train['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
    tfidf_f = TfidfVectorizer(stop_words=['central_a', 'pre', 'war','features'], max_features=98)
    train_sparse_f = tfidf_f.fit_transform(train['features_modify'])
    feature_list = tfidf_f.get_feature_names()
    array1 = train_sparse_f.toarray()
    for i, feature_str in enumerate(feature_list):
        list_all = []
        [rows, cols] = array1.shape
        for row in range(rows):
            list_all.append(array1[row][i])
        feature_str = feature_str + '(f)'
        feature_v = pd.Series(list_all,train.index, name = feature_str)
        train[feature_str] = feature_v
    #set all feature words in tf_idf as features for train for description_modify
    train['description_modify'] = train['description'].apply(lambda x:valid_des(x))
    tfidf_d = TfidfVectorizer(stop_words=['features'], max_features=100)
    train_sparse_d = tfidf_d.fit_transform(train['description_modify'])
    description_list = tfidf_d.get_feature_names()
    array2 = train_sparse_d.toarray()
    for i, description_str in enumerate(description_list):
        list_all = []
        [rows, cols] = array2.shape
        for row in range(rows):
            list_all.append(array2[row][i])
        description_str_ = description_str + '(d)'
        description_v = pd.Series(list_all,train.index, name = description_str_)
        train[description_str_] = description_v
    ## /*add something to change the created to year month day*/
    train['created']=pd.to_datetime(train["created"])
    #extract pieces
    train['year']=train['created'].dt.year
    train['month']=train['created'].dt.month
    train['day']=train['created'].dt.day
    ## /*add something to do feature selection*/
    train = train.drop(["building_id", "created", "description", "display_address", "features", 
                         "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify"], axis=1)
    #drop stop words
    train = train.drop(['an(d)','and(d)','are(d)','as(d)','at(d)','be(d)','by(d)','can(d)','for(d)','from(d)','in(d)','is(d)','it(d)','just(d)','me(d)','more(d)','natural(d)','new(d)','or(d)','park(d)','that(d)','the(d)','this(d)','to(d)','will(d)','with(d)','you(d)','your(d)'], axis = 1)
    return train

In [3]:
#main function
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [4]:
train['test']=False
test['test']=True

cdf = train.append(test).reset_index(drop=True)
cdf = dataforclf(cdf)
train = cdf[cdf.test == False]
test = cdf[cdf.test == True]
train = train.drop(["test"],axis=1)
test = test.drop(["interest_level","test"],axis=1)

E:\Anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
E:\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
test

,bathrooms,bedrooms,listing_id,price_modify,latitude_modify,longitude_modify,photos_number,_photos(f),actual_apt(f),balcony(f),...,supports(d),text(d),two(d),unit(d),west(d),windows(d),york(d),year,month,day
49352,1.0,1,7142618,2950.0,40.7185,-73.9865,8,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,11
49353,1.0,2,7210040,2850.0,40.7278,-74.0000,3,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,24
49354,1.0,0,7174566,2295.0,40.7260,-74.0026,1,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,17
49355,1.0,2,7191391,2900.0,40.7321,-74.0028,4,0.0,0.0,0.0,...,0.330474,0.000000,0.0,0.000000,0.0,0.0,0.292075,2016,6,21
49356,1.0,1,7171695,3254.0,40.7054,-74.0095,6,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124006,1.0,1,6928108,1700.0,40.7925,-73.9454,10,0.0,0.0,0.0,...,0.000000,0.273799,0.0,0.000000,0.0,0.0,0.000000,2016,4,26
124007,1.0,2,6906674,4195.0,40.7456,-73.9797,4,0.0,0.0,0.0,...,0.000000,0.188096,0.0,0.000000,0.0,0.0,0.000000,2016,4,21
124008,1.0,0,6897967,2400.0,40.7416,-73.9829,0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.135853,0.0,0.0,0.000000,2016,4,20
124009,2.0,2,6842183,6895.0,40.7485,-73.9800,8,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.193046,2016,4,8


In [6]:
train

,bathrooms,bedrooms,interest_level,listing_id,price_modify,latitude_modify,longitude_modify,photos_number,_photos(f),actual_apt(f),...,supports(d),text(d),two(d),unit(d),west(d),windows(d),york(d),year,month,day
0,1.0,1,medium,7170325,2400.0,40.7108,-73.9539,12,0.0,0.0,...,0.000000,0.277420,0.0,0.000000,0.0,0.0,0.000000,2016,6,16
1,1.0,2,low,7092344,3800.0,40.7513,-73.9722,6,0.0,0.0,...,0.000000,0.158305,0.0,0.000000,0.0,0.0,0.000000,2016,6,1
2,1.0,2,medium,7158677,3495.0,40.7575,-73.9625,6,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,14
3,1.5,3,medium,7211212,3000.0,40.7145,-73.9425,5,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,24
4,1.0,0,low,7225292,2795.0,40.7439,-73.9743,4,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,3,low,6824800,2800.0,40.8433,-73.9396,5,0.0,0.0,...,0.125354,0.000000,0.0,0.000000,0.0,0.0,0.110788,2016,4,5
49348,1.0,2,medium,6813268,2395.0,40.8198,-73.9578,5,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.228953,2016,4,2
49349,1.0,1,medium,6927093,1850.0,40.5765,-73.9554,3,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,4,26
49350,1.0,2,medium,6892816,4195.0,40.7448,-74.0017,5,0.0,0.0,...,0.000000,0.000000,0.0,0.163351,0.0,0.0,0.000000,2016,4,19


# Feature selection

In [7]:
#For training
x_train = train.drop(["listing_id","interest_level","photos_number","day"],axis=1)
#For testing
x_test = test.drop(["listing_id","photos_number","day"],axis=1)

y = pd.get_dummies(train['interest_level'])
#X_train,X_test,Y_train,Y_test = train_test_split(x_train,y,test_size=0.33,random_state=0)

In [8]:
fsclf = ExtraTreesClassifier()
fsclf = fsclf.fit(x_train,y)
names = x_train.columns
sorted_features = sorted(zip(fsclf.feature_importances_,names),reverse=True)
print(sorted_features)


E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[(0.06264552108862122, 'price_modify'), (0.05433114620409368, 'month'), (0.028316796841411536, 'latitude_modify'), (0.02819259077761866, 'longitude_modify'), (0.023384615991477307, 'bedrooms'), (0.017576431839107797, 'hardwood_floors(f)'), (0.015841120461787917, 'no_fee(f)'), (0.014685347790814878, 'of(d)'), (0.014393894932789416, 'apartment(d)'), (0.013083141254493127, 'elevator(f)'), (0.012911065700805932, 'bedroom(d)'), (0.012891202917291436, 'cats_allowed(f)'), (0.012636134613008568, 'kitchen(d)'), (0.012503050959741377, 'hardwood(d)'), (0.012486019264958027, 'on(d)'), (0.012385557998509997, 'dogs_allowed(f)'), (0.01237523544180042, 'laundry_in_building(f)'), (0.012343303181231047, 'doorman(f)'), (0.011699163735173696, 'renovated(d)'), (0.011233296486394293, 'building(d)'), (0.010867206054537688, 'dishwasher(f)'), (0.010840097363398596, 'all(d)'), (0.010795893577933782, 'great(d)'), (0.010392967444551883, 'large(d)'), (0.00994916019269933, 'living(d)'), (0.009929275521274495, 'stee

In [9]:
feature_selected = [sorted_features[i][1] for i in range (100)]
feature_selected

['price_modify',
 'month',
 'latitude_modify',
 'longitude_modify',
 'bedrooms',
 'hardwood_floors(f)',
 'no_fee(f)',
 'of(d)',
 'apartment(d)',
 'elevator(f)',
 'bedroom(d)',
 'cats_allowed(f)',
 'kitchen(d)',
 'hardwood(d)',
 'on(d)',
 'dogs_allowed(f)',
 'laundry_in_building(f)',
 'doorman(f)',
 'renovated(d)',
 'building(d)',
 'dishwasher(f)',
 'all(d)',
 'great(d)',
 'large(d)',
 'living(d)',
 'steel(d)',
 'stainless(d)',
 'call(d)',
 'high(d)',
 'laundry(d)',
 'located(d)',
 'contact(d)',
 'spacious(d)',
 'beautiful(d)',
 'bathrooms',
 'email(d)',
 'one(d)',
 'has(d)',
 'text(d)',
 'no(d)',
 'room(d)',
 'closet(d)',
 'close(d)',
 'full(d)',
 'unit(d)',
 'granite(d)',
 'space(d)',
 'east(d)',
 'floors(d)',
 'studio(d)',
 'private(d)',
 'marble(d)',
 'bathroom(d)',
 'size(d)',
 'amazing(d)',
 'steps(d)',
 'schedule(d)',
 'windows(d)',
 'fitness_center(f)',
 'have(d)',
 'laundry_in_unit(f)',
 'appliances(d)',
 'heart(d)',
 'available(d)',
 'please(d)',
 'two(d)',
 'central(d)',
 'fr

In [10]:
#For training
x_train = train[feature_selected]
#For testing
x_test = test[feature_selected]
x_train

,price_modify,month,latitude_modify,longitude_modify,bedrooms,hardwood_floors(f),no_fee(f),of(d),apartment(d),elevator(f),...,housing(d),supports(d),equal(d),high_speed_internet(f),furnished(f),loft(f),balcony(f),common_outdoor_space(f),swimming_pool(f),new_construction(f)
0,2400.0,6,40.7108,-73.9539,1,0.332455,0.000000,0.132806,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,3800.0,6,40.7513,-73.9722,2,0.383705,0.441931,0.227351,0.088526,0.360138,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,3495.0,6,40.7575,-73.9625,2,0.357887,0.000000,0.076161,0.355867,0.335906,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,3000.0,6,40.7145,-73.9425,3,0.000000,0.000000,0.113306,0.132358,0.000000,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,2795.0,6,40.7439,-73.9743,0,0.000000,0.000000,0.000000,0.000000,0.416537,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,2800.0,4,40.8433,-73.9396,3,0.574134,0.000000,0.239748,0.000000,0.538871,...,0.124781,0.125354,0.12399,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
49348,2395.0,4,40.8198,-73.9578,2,0.000000,0.310128,0.123864,0.144691,0.252729,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.716101,0.0,0.0
49349,1850.0,4,40.5765,-73.9554,1,0.220437,0.253887,0.115974,0.135475,0.206897,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
49350,4195.0,4,40.7448,-74.0017,2,0.000000,0.400902,0.086034,0.100500,0.000000,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


# First Version of Bagging

In [8]:
Best_validation_score = 0
kf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=None)
best_loss=999
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(BaggingClassifier(base_estimator = tree.DecisionTreeClassifier()))
    Tree.fit(X_train,Y_train)
    
    print("Current training accuracy: %0.5f" % Tree.score(X_train,Y_train))
    Validation_score = Tree.score(X_test,Y_test)
    if Validation_score > Best_validation_score:
        Best_validation_score = Validation_score
        Best_Tree = Tree
    Y_labeled = Tree.predict_proba(X_test)
    loss = log_loss(Y_test,Y_labeled)
    if best_loss > loss:
        best_loss = loss
    print ("Current validation accuracy: %0.5f" % Validation_score)
print ("Best validation accuracy: %0.5f" % Best_validation_score)
print ("Best log loss: %0.5f" % best_loss)

Current training accuracy: 0.94178
Current validation accuracy: 0.61973
Current training accuracy: 0.94173
Current validation accuracy: 0.60717
Current training accuracy: 0.94081
Current validation accuracy: 0.63100
Current training accuracy: 0.94124
Current validation accuracy: 0.62594
Current training accuracy: 0.94176
Current validation accuracy: 0.61398
Current training accuracy: 0.93890
Current validation accuracy: 0.62614
Current training accuracy: 0.94061
Current validation accuracy: 0.62573
Current training accuracy: 0.94002
Current validation accuracy: 0.63060
Current training accuracy: 0.94151
Current validation accuracy: 0.62817
Current training accuracy: 0.94101
Current validation accuracy: 0.62472
Current training accuracy: 0.93941
Current validation accuracy: 0.61386
Current training accuracy: 0.94013
Current validation accuracy: 0.61791
Current training accuracy: 0.93975
Current validation accuracy: 0.61520
Current training accuracy: 0.94108
Current validation accuracy: 

In [9]:
Y_Prediction = Best_Tree.predict_proba(x_test)
test['listing_id'] = test['listing_id'].astype(int)
ids = test['listing_id']
labels = pd.DataFrame(Y_Prediction)
labels.reset_index(drop=True, inplace=True)
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, labels], axis=1, ignore_index=True)
out.columns = ['listing_id','high','low','medium']
out

,listing_id,high,low,medium
0,7142618,0.0,0.9,0.30
1,7210040,0.8,0.2,0.30
2,7174566,0.0,0.8,0.10
3,7191391,0.4,0.7,0.20
4,7171695,0.1,0.8,0.10
...,...,...,...,...
74654,6928108,0.3,0.5,0.50
74655,6906674,0.0,0.4,0.90
74656,6897967,0.0,1.0,0.10
74657,6842183,0.0,1.0,0.00


In [10]:
out.to_csv('submission.csv',index = False)

# Frist Version of decision tree classifier

In [8]:
Best_validation_score = 0
kf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=None)
best_loss=999
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier())
    Tree.fit(X_train,Y_train)
    print("Current training accuracy: %0.5f" % Tree.score(X_train,Y_train))
    Validation_score = Tree.score(X_test,Y_test)
    if Validation_score > Best_validation_score:
        Best_validation_score = Validation_score
        Best_Tree = Tree
    Y_labeled = Tree.predict_proba(X_test)
    loss = log_loss(Y_test,Y_labeled)
    if best_loss > loss:
        best_loss = loss
    print ("Current validation accuracy: %0.5f" % Validation_score)
print ("Best validation accuracy: %0.5f" % Best_validation_score)
print ("Best log loss: %0.5f" % best_loss)

Current training accuracy: 0.97334
Current validation accuracy: 0.53241
Current training accuracy: 0.97373
Current validation accuracy: 0.54113
Current training accuracy: 0.97278
Current validation accuracy: 0.55218
Current training accuracy: 0.97341
Current validation accuracy: 0.53212
Current training accuracy: 0.97442
Current validation accuracy: 0.53171
Current training accuracy: 0.97382
Current validation accuracy: 0.53556
Current training accuracy: 0.97438
Current validation accuracy: 0.54225
Current training accuracy: 0.97249
Current validation accuracy: 0.54853
Current training accuracy: 0.97321
Current validation accuracy: 0.55502
Current training accuracy: 0.97384
Current validation accuracy: 0.53698
Current training accuracy: 0.97267
Current validation accuracy: 0.53890
Current training accuracy: 0.97339
Current validation accuracy: 0.52938
Current training accuracy: 0.97373
Current validation accuracy: 0.54610
Current training accuracy: 0.97283
Current validation accuracy: 

In [9]:
Y_Prediction = Best_Tree.predict_proba(x_test)
test['listing_id'] = test['listing_id'].astype(int)
ids = test['listing_id']
labels = pd.DataFrame(Y_Prediction)
labels.reset_index(drop=True, inplace=True)
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, labels], axis=1, ignore_index=True)
out.columns = ['listing_id','high','low','medium']
out

,listing_id,high,low,medium
0,7142618,0.0,1.0,0.0
1,7210040,1.0,1.0,0.0
2,7174566,0.0,1.0,0.0
3,7191391,0.0,0.0,0.0
4,7171695,0.0,1.0,0.0
...,...,...,...,...
74654,6928108,0.0,0.0,0.0
74655,6906674,0.0,0.0,1.0
74656,6897967,0.0,1.0,0.0
74657,6842183,0.0,1.0,0.0


In [10]:
out.to_csv('submission.csv',index = False)

# Improvement on parameters of the decision tree and k-fold cross validation

In [8]:
#Choose the best k for k-fold cross validation.
best_loss = 999
for i in range(2,11):
    kf = RepeatedKFold(n_splits=i, n_repeats=1, random_state=None)
    for train_index, test_index in kf.split(x_train):
        x_train['id'] = list(range(0,49352))
        x_train.set_index("id", inplace=True)
        y['id'] = list(range(0,49352))
        y.set_index("id", inplace=True)
        X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
        Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
        Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0))
        Tree.fit(X_train,Y_train)
        Y_labeled = Tree.predict_proba(X_test)
        loss = log_loss(Y_test,Y_labeled)
        if best_loss > loss:
            best_loss = loss
            res = i
print ("Best result: %d" % res)

Best result: 10


In [8]:
#Choose the best max_depth.
best_loss=999
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for train_index, test_index in kf.split(x_train):
    x_train['id']=list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id']=list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    for i in range(2,15):
        Tree=OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=i,random_state=0))
        Tree.fit(X_train,Y_train)
        #validation_score = Tree.score(X_test,Y_test)
        Y_labeled=Tree.predict_proba(X_test)
        loss=log_loss(Y_test,Y_labeled)
        if best_loss>loss:
            best_loss=loss
            best_depth = i
print ("Best result: %d" % best_depth)

Best result: 6


In [8]:
#Choose the best max_leaf_nodes.
best_loss = 999
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for i in range (15,30):
    for train_index, test_index in kf.split(x_train):
        x_train['id'] = list(range(0,49352))
        x_train.set_index("id", inplace=True)
        y['id'] = list(range(0,49352))
        y.set_index("id", inplace=True)
        X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
        Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
        Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=0,max_leaf_nodes=i))
        Tree.fit(X_train,Y_train)
        #validation_score = Tree.score(X_test,Y_test)
        Y_labeled = Tree.predict_proba(X_test)
        loss = log_loss(Y_test,Y_labeled)
        if best_loss > loss:
            best_loss = loss
            best_res = i
print ("Best result: %d" % best_res)

Best result: 23


In [8]:
#Choose the best min_samples_leaf.
best_loss=999
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for i in range (2,20):
    for train_index, test_index in kf.split(x_train):
        x_train['id']=list(range(0,49352))
        x_train.set_index("id", inplace=True)
        y['id']=list(range(0,49352))
        y.set_index("id", inplace=True)
        X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
        Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
        Tree=OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=0,max_leaf_nodes=23,min_samples_leaf=i))
        Tree.fit(X_train,Y_train)
        #validation_score = Tree.score(X_test,Y_test)
        Y_labeled=Tree.predict_proba(X_test)
        loss=log_loss(Y_test,Y_labeled)
        if best_loss>loss:
            best_loss=loss
            best_res = i
print ("Best result: %d" % best_res)

Best result: 6


In [13]:
#Choose the best criterion. The result shows the two criterion has similar effects.
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='entropy',max_depth=6,random_state=0)).fit(X_train,Y_train)
    validation_score1 = Tree.score(X_test,Y_test)
    Y_labeled = Tree.predict_proba(X_test)
    loss1 = log_loss(Y_test,Y_labeled)
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=6,random_state=0)).fit(X_train,Y_train)
    validation_score2 = Tree.score(X_test,Y_test)
    Y_labeled = Tree.predict_proba(X_test)
    loss2 = log_loss(Y_test,Y_labeled)
    if validation_score1 >= validation_score2:
        print('Validation accuray: Entropy')
    else:
        print('Validation accuracy: Gini index')
    if loss1 <= loss2:
        print('Log loss: Entropy')
    else:
        print('Log loss: Gini index')


Validation accuracy: Gini index
Log loss: Gini index
Validation accuray: Entropy
Log loss: Gini index
Validation accuracy: Gini index
Log loss: Entropy
Validation accuracy: Gini index
Log loss: Gini index
Validation accuracy: Gini index
Log loss: Gini index
Validation accuray: Entropy
Log loss: Gini index
Validation accuracy: Gini index
Log loss: Gini index
Validation accuray: Entropy
Log loss: Gini index


# Final version of decision tree

In [8]:
best_loss = 999
Best_validation_score = 0
kf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=None)
best_precision=0
precision=[]
recall=[]
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=6,random_state=0,splitter='best',max_leaf_nodes=23,min_samples_leaf=6))
    Tree.fit(X_train,Y_train)
    print("Current training accuracy: %0.5f" % Tree.score(X_train,Y_train))
    #accuracy
    Validation_score = Tree.score(X_test,Y_test)
    if Validation_score > Best_validation_score:
        Best_validation_score = Validation_score
        #Best_Tree = Tree   
    #log loss
    Y_labeled = Tree.predict_proba(X_test)
    loss = log_loss(Y_test,Y_labeled)
    if best_loss > loss:
        best_loss = loss
        #Best_Tree = Tree  
    #precision
    Y_Pred = Tree.predict(X_test)
    print(multilabel_confusion_matrix(Y_test, Y_Pred))
    if best_precision < precision_score(Y_test, Y_Pred, average='macro'):
        best_precision = precision_score(Y_test, Y_Pred, average='macro')
        Best_Tree = Tree  
    precision.append(precision_score(Y_test, Y_Pred, average='macro'))
    recall.append(recall_score(Y_test, Y_Pred, average='macro'))
    print ("Current validation accuracy: %0.5f" % Validation_score)
print ("Best validation accuracy: %0.5f" % Best_validation_score)
print ("Best log loss: %0.5f" % best_loss)

#Y_pred = Best_Tree.predict(X_test)
#multilabel_confusion_matrix(Y_test, Y_pred)   
print(precision)
print(recall)


Current training accuracy: 0.63912
[[[4518   32]
  [ 338   48]]

 [[ 528  948]
  [ 315 3145]]

 [[3846    0]
  [1090    0]]]
Current validation accuracy: 0.64587


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.61726
[[[4500   17]
  [ 389   30]]

 [[ 578  982]
  [ 416 2960]]

 [[3793    2]
  [1139    2]]]
Current validation accuracy: 0.60555
Current training accuracy: 0.60981
[[[4535   44]
  [ 311   45]]

 [[ 621  852]
  [ 478 2984]]

 [[3818    0]
  [1117    0]]]
Current validation accuracy: 0.61317


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.61040
[[[4515   19]
  [ 355   46]]

 [[ 688  851]
  [ 445 2951]]

 [[3780   17]
  [1122   16]]]
Current validation accuracy: 0.61033
Current training accuracy: 0.64658
[[[4516   45]
  [ 344   30]]

 [[ 511  980]
  [ 309 3135]]

 [[3794   24]
  [1101   16]]]
Current validation accuracy: 0.64296
Current training accuracy: 0.59529
[[[4520   30]
  [ 340   45]]

 [[ 699  775]
  [ 550 2911]]

 [[3846    0]
  [1089    0]]]
Current validation accuracy: 0.59838


...

Current training accuracy: 0.64453
[[[4503   46]
  [ 339   47]]

 [[ 486 1048]
  [ 301 3100]]

 [[3785    2]
  [1145    3]]]
Current validation accuracy: 0.63627
Current training accuracy: 0.61823
[[[4496   36]
  [ 370   33]]

 [[ 614  896]
  [ 439 2986]]

 [[3813   15]
  [1092   15]]]
Current validation accuracy: 0.61317
Current training accuracy: 0.59966
[[[4521   24]
  [ 349   41]]

 [[ 665  853]
  [ 493 2924]]

 [[3807    0]
  [1128    0]]]
Current validation accuracy: 0.60061


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.60220
[[[4488   28]
  [ 359   60]]

 [[ 708  804]
  [ 514 2909]]

 [[3842    0]
  [1093    0]]]
Current validation accuracy: 0.60020


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.61118
[[[4551   33]
  [ 323   28]]

 [[ 626  882]
  [ 520 2907]]

 [[3746   32]
  [1132   25]]]
Current validation accuracy: 0.59635
Current training accuracy: 0.64273
[[[4539   16]
  [ 346   34]]

 [[ 523  999]
  [ 327 3086]]

 [[3774   19]
  [1123   19]]]
Current validation accuracy: 0.63607
Best validation accuracy: 0.65836
Best log loss: 0.66607
[0.45612834921410544, 0.6297285821719903, 0.4278372055785326, 0.6562370763843673, 0.5206156338598623, 0.46324832700307467, 0.5206240086396003, 0.494713608957795, 0.6341386765888979, 0.4582446007571847, 0.7123294346978558, 0.450214618388592, 0.6031131572092397, 0.5832565914444191, 0.5702187540783402, 0.43806863159239917, 0.6101450699151848, 0.6452931173863351, 0.46673025854328204, 0.5922847866334692, 0.6307021136699625, 0.46793794776186565, 0.5486022710803085, 0.4274205613555793, 0.48082881884593287, 0.6374105197634609, 0.6149253292920438, 0.5554205583162056, 0.4236401429338814, 0.4124924019699056, 0.567132568548

In [9]:
Y_Prediction=Best_Tree.predict_proba(x_test)
test['listing_id'] = test['listing_id'].astype(int)
ids = test['listing_id']
labels = pd.DataFrame(Y_Prediction)
labels.reset_index(drop=True, inplace=True)
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, labels], axis=1, ignore_index=True)
out.columns=['listing_id','high','low','medium']
out

,listing_id,high,low,medium
0,7142618,0.010679,0.851006,0.227256
1,7210040,0.041707,0.435088,0.155424
2,7174566,0.055496,0.664519,0.195402
3,7191391,0.041707,0.435088,0.384275
4,7171695,0.010679,0.851006,0.231205
...,...,...,...,...
74654,6928108,0.190401,0.341701,0.294811
74655,6906674,0.104250,0.561509,0.267947
74656,6897967,0.055496,0.664519,0.195402
74657,6842183,0.041707,0.886485,0.084235


In [10]:
out.to_csv('submission.csv',index = False)

# improvements on parameters of bagging

In [11]:
# specify number of folds for k-fold CV
n_folds = 10

# instantiate the model (note we are specifying a max_depth)
model_to_set = OneVsRestClassifier(BaggingClassifier())
# parameters to build the model on
parameters = {
    "estimator__bootstrap":['True','False']
}
# fit tree on training data
HGB = GridSearchCV(model_to_set, parameters,cv=n_folds, scoring="neg_log_loss")

HGB.fit(x_train, y)
HGB.best_params_

{'estimator__bootstrap': 'False'}

In [8]:
# specify number of folds for k-fold CV
n_folds = 10

# instantiate the model (note we are specifying a max_depth)
model_to_set = OneVsRestClassifier(BaggingClassifier())
# parameters to build the model on
parameters = {
    "estimator__max_samples":[0.2,0.4,0.6,0.8,1.0]
}
# fit tree on training data
HGB = GridSearchCV(model_to_set, parameters,cv=n_folds, scoring="neg_log_loss")

HGB.fit(x_train, y)
HGB.best_params_

{'estimator__max_samples': 0.2}

# Final version of Bagging

In [11]:
best_loss = 999
Best_validation_score = 0
kf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=None)
best_precision=0
precision=[]
recall=[]
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(BaggingClassifier(base_estimator = tree.DecisionTreeClassifier(criterion='gini',max_depth=6,random_state=0,splitter='best',max_leaf_nodes=23,min_samples_leaf=6),n_estimators=9,random_state=0,max_samples=0.8,bootstrap='False',max_features=1.0))
    Tree.fit(X_train,Y_train)
    print("Current training accuracy: %0.5f" % Tree.score(X_train,Y_train))
    #accuracy
    Validation_score = Tree.score(X_test,Y_test)
    if Validation_score > Best_validation_score:
        Best_validation_score = Validation_score
        #Best_Tree = Tree   
    #log loss
    Y_labeled = Tree.predict_proba(X_test)
    loss = log_loss(Y_test,Y_labeled)
    if best_loss > loss:
        best_loss = loss
        #Best_Tree = Tree  
    #precision
    Y_Pred = Tree.predict(X_test)
    print(multilabel_confusion_matrix(Y_test, Y_Pred))
    if best_precision < precision_score(Y_test, Y_Pred, average='macro'):
        best_precision = precision_score(Y_test, Y_Pred, average='macro')
        Best_Tree = Tree  
    precision.append(precision_score(Y_test, Y_Pred, average='macro'))
    recall.append(recall_score(Y_test, Y_Pred, average='macro'))
    print ("Current validation accuracy: %0.5f" % Validation_score)
print ("Best validation accuracy: %0.5f" % Best_validation_score)
print ("Best log loss: %0.5f" % best_loss)

#Y_pred = Best_Tree.predict(X_test)
#multilabel_confusion_matrix(Y_test, Y_pred)   
print(precision)
print(recall)


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Current training accuracy: 0.64936
[[[4524   12]
  [ 366   34]]

 [[ 477 1064]
  [ 253 3142]]

 [[3795    0]
  [1141    0]]]
Current validation accuracy: 0.64344


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Current training accuracy: 0.64909
[[[4544   23]
  [ 334   35]]

 [[ 459 1013]
  [ 240 3224]]

 [[3833    0]
  [1103    0]]]
Current validation accuracy: 0.65964


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


...

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Current training accuracy: 0.64590
[[[4503   24]
  [ 369   39]]

 [[ 504 1020]
  [ 299 3112]]

 [[3819    0]
  [1116    0]]]
Current validation accuracy: 0.63850


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Current training accuracy: 0.64955
[[[4559   20]
  [ 322   34]]

 [[ 440 1003]
  [ 259 3233]]

 [[3848    0]
  [1087    0]]]
Current validation accuracy: 0.66160
Best validation accuracy: 0.66322
Best log loss: 0.65560
[0.4953861633139683, 0.4547880060441811, 0.44114791483212534, 0.432860738045997, 0.8484665556165777, 0.5432440675118431, 0.7981459061967536, 0.465224737701362, 0.4933796756041046, 0.6038628591780121, 0.4770544396620437, 0.4540107152223596, 0.4643292487329185, 0.5543420527195257, 0.4507823180920553, 0.4131830849388101, 0.43602744991633885, 0.44666395075140325, 0.4467639546239952, 0.4532911669296249, 0.45704043575089215, 0.44376895893923757, 0.44194190743677125, 0.47912305341353756, 0.4582444779396713, 0.4198701606382267, 0.48718142337412895, 0.43555290997215357, 0.4554886702882592, 0.6089962596996595, 0.4171428571428571, 0.5743845984121069, 0.4775095979834168, 0.4348607785758624, 0.44888093127125234, 0.44077992569371877, 0.4368163362789587, 0.48327399319566416, 0.46164480

E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
Y_Prediction = Best_Tree.predict_proba(x_test)
test['listing_id'] = test['listing_id'].astype(int)
ids = test['listing_id']
labels = pd.DataFrame(Y_Prediction)
labels.reset_index(drop=True, inplace=True)
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, labels], axis=1, ignore_index=True)
out.columns = ['listing_id','high','low','medium']
out

,listing_id,high,low,medium
0,7142618,0.027461,0.819462,0.230789
1,7210040,0.049631,0.476905,0.165529
2,7174566,0.050469,0.741102,0.239851
3,7191391,0.049631,0.467351,0.356912
4,7171695,0.027461,0.866177,0.181708
...,...,...,...,...
74654,6928108,0.188940,0.580214,0.230949
74655,6906674,0.084479,0.644613,0.287932
74656,6897967,0.042562,0.741102,0.206319
74657,6842183,0.030006,0.884559,0.061985


In [13]:
out.to_csv('submission.csv',index = False)